# RAG Chatbot (Pinecone-only, OpenAI LLM)

이 노트북은 웹 없이도 로컬에서 챗봇 동작을 확인할 수 있는 전용 런타임입니다.

## ⚠️ 중요: 실행 전 필수 확인사항
1. **커널 재시작**: `Ctrl+Shift+P` → "Jupyter: Restart Kernel" 실행
2. **순차 실행**: Cell 1 → Cell 2 → Cell 3 → Cell 4 → Cell 5 순서대로 실행
3. **중복 실행 금지**: Cell 5를 여러 번 실행하지 마세요 (while 루프가 중복 실행됨)

실행 순서: 1) 환경/설치 → 2) Pinecone 연결 → 3) 검색 함수 → 4) LLM 챗봇

In [1]:
# 환경/설치
import sys, platform
print('Python:', sys.version)
print('Platform:', platform.platform())

!{sys.executable} -m pip install -q --upgrade pip
!{sys.executable} -m pip install -q "pinecone>=5.0.0" sentence-transformers rank-bm25 pyyaml openai


Python: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
Platform: Windows-10-10.0.26100-SP0


In [2]:
# Pinecone 연결
import os
from pinecone import Pinecone
from config import PINECONE_API_KEY, PINECONE_INDEX_NAME

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)
print(index.describe_index_stats())


{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 1082}},
 'total_vector_count': 1082,
 'vector_type': 'dense'}


In [3]:
# 검색 함수 (Cell 26 요약)
import re, numpy as np
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from config import EMBEDDING_MODEL_NAME

model = SentenceTransformer(EMBEDDING_MODEL_NAME, device="cpu")

def simple_tokenize(s: str):
    return re.findall(r"[A-Za-z0-9가-힣]+", (s or "").lower())

def vector_search(query: str, top_k: int = 50, meta_filter=None):
    q_vec = model.encode([f"query: {query}"], convert_to_numpy=True, normalize_embeddings=True)[0]
    kwargs = {"vector": q_vec.tolist(), "top_k": top_k, "include_values": False, "include_metadata": True}
    if meta_filter:
        kwargs["filter"] = meta_filter
    res = index.query(**kwargs)
    return [(m["id"], float(m["score"]), m.get("metadata", {})) for m in res.get("matches", [])]

def bm25_over_candidates(query: str, candidates):
    ids, docs = [], []
    for cid, _, meta in candidates:
        text = (meta or {}).get("text_content") or ""
        if not text:
            title = (meta or {}).get("title") or ""
            keywords = (meta or {}).get("keywords") or ""
            text = f"{title}\n{keywords}"
        ids.append(cid)
        docs.append(simple_tokenize(text))
    if not docs:
        return {}
    bm25 = BM25Okapi(docs)
    scores = bm25.get_scores(simple_tokenize(query)) if query else np.zeros(len(ids))
    max_b = float(np.max(scores)) if len(scores) else 0.0
    return {ids[i]: (float(scores[i])/max_b if max_b>0 else 0.0) for i in range(len(ids))}


In [ ]:
# 개선된 LLM 챗봇
from collections import deque
from typing import List, Dict, Tuple
from openai import OpenAI
from config import OPENAI_API_KEY, LLM_MODEL_NAME, DEFAULT_VECTOR_WEIGHT, DEFAULT_BM25_WEIGHT, DEFAULT_TOP_K, DEFAULT_CONTEXT_CHARS, DEFAULT_CONTEXT_TOP_N
import os

openai_client = OpenAI(api_key=OPENAI_API_KEY)

# 개선된 시스템 프롬프트
SYSTEM_PROMPT = (
    "너는 RAG 기반 도우미야. 제공된 컨텍스트를 우선 활용해서 간결하고 정확하게 답해.\n"
    "근거가 없으면 솔직히 모른다고 말해.\n"
    "출처를 bullet로 함께 제공해."
    )

def build_context_improved(query: str, candidates: List[Tuple[str, float, Dict]], vec_w: float, bm25_w: float, top_n: int, max_chars: int):
    """개선된 컨텍스트 빌딩"""
    bm25_scores = bm25_over_candidates(query, candidates)
    scored = []
    for cid, v_score, meta in candidates:
        b_score = bm25_scores.get(cid, 0.0)
        combo = vec_w * float(v_score) + bm25_w * float(b_score)
        scored.append((combo, cid, meta))
    scored.sort(reverse=True, key=lambda x: x[0])

    picked, used = [], 0
    for _, cid, meta in scored[: max(1, int(top_n) * 3)]:
        text = (meta or {}).get("text_content") or (meta or {}).get("title") or ""
        if not text:
            continue
        if used + len(text) > max_chars:
            continue
        picked.append({
            "id": cid,
            "title": (meta or {}).get("title"),
            "source": (meta or {}).get("source_doc"),
            "chunk": text,
        })
        used += len(text)
        if len(picked) >= int(top_n):
            break
    return picked

def call_openai_improved(api_key: str, model: str, messages: List[Dict]) -> str:
    """개선된 OpenAI 호출"""
    if not api_key:
        raise ValueError("OpenAI API 키가 필요합니다.")
    client = OpenAI(api_key=api_key)
    r = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.2,
    )
    return r.choices[0].message.content or ""

_history_deque = deque(maxlen=3)

def build_messages_with_history(question: str, context_text: str, citations: str) -> List[Dict[str, str]]:
    """최근 대화 몇 턴을 포함하도록 메시지를 구성"""
    messages: List[Dict[str, str]] = [{"role": "system", "content": SYSTEM_PROMPT}]
    for past_question, past_answer, _ in _history_deque:
        if past_question:
            messages.append({"role": "user", "content": past_question})
        if past_answer:
            messages.append({"role": "assistant", "content": past_answer})
    user_content = (
        f"질문: {question}\n\n컨텍스트:\n{context_text}\n\n"
        "컨텍스트를 기반으로 답하세요. 답 끝에 '출처' 섹션을 넣어 아래 목록에서 근거를 인용하세요.\n"
        f"출처 목록:\n{citations}"
    )
    messages.append({"role": "user", "content": user_content})
    return messages

def chat_once_improved(question: str, vec_w=DEFAULT_VECTOR_WEIGHT, bm25_w=DEFAULT_BM25_WEIGHT, top_k=DEFAULT_TOP_K, ctx_n=DEFAULT_CONTEXT_TOP_N, max_ctx_chars=DEFAULT_CONTEXT_CHARS, debug=True):
    """개선된 챗봇 로직"""
    if debug:
        print("="*90)
        print(f"[질문] {question}")

    # 벡터 검색
    candidates = vector_search(question, top_k=top_k)
    
    if debug:
        print(f"[벡터 후보 수] {len(candidates)}")
        print("-"*90)
        print("[Vector Top 10]")
        for i, (cid, vscore, meta) in enumerate(candidates[:10], 1):
            print(f"  {i:>2}. {vscore:.4f} | {(meta or {}).get('title','N/A')} | {cid}")

    # 개선된 컨텍스트 빌딩
    contexts = build_context_improved(question, candidates, vec_w, bm25_w, ctx_n, max_ctx_chars)
    
    if debug:
        print("-"*90)
        print(f"[선택된 컨텍스트 수] {len(contexts)}")
        for i, ctx in enumerate(contexts, 1):
            print(f"  {i:>2}. {ctx['title'] or 'N/A'} | {ctx['id']}")

    # 컨텍스트 텍스트 구성
    context_text = "\n\n".join([f"[#{i+1}] {c['chunk']}" for i, c in enumerate(contexts)])
    citations = "\n".join(
        [f"- [#{i+1}] {c.get('title') or c.get('source') or c['id']}" for i, c in enumerate(contexts)]
    )

    if debug:
        print("-"*90)
        print(f"[컨텍스트 길이] {len(context_text)}")
        print("="*90)

    # LLM 호출
    messages = build_messages_with_history(question, context_text, citations)

    answer = call_openai_improved(
        OPENAI_API_KEY,
        LLM_MODEL_NAME,
        messages,
    )

    # 히스토리 저장
    _history_deque.append((question, answer, ""))
    
    return answer.strip()

# ========================================
# 중복 실행 방지 체크
# ========================================
_CHATBOT_PID_FILE = ".chatbot_running.pid"

if os.path.exists(_CHATBOT_PID_FILE):
    print("⚠️  경고: 챗봇이 이미 실행 중일 수 있습니다!")
    print("⚠️  다른 커널에서 실행 중이거나 이전 실행이 완전히 종료되지 않았습니다.")
    print("⚠️  해결 방법:")
    print("   1. Ctrl+Shift+P → 'Jupyter: Restart Kernel' 실행")
    print("   2. 모든 셀을 순차적으로 다시 실행 (1→2→3→4→5)")
    raise RuntimeError("중복 실행 방지: 커널을 재시작하세요")

# PID 파일 생성
with open(_CHATBOT_PID_FILE, "w") as f:
    f.write(str(os.getpid()))

print("✅ 개선된 챗봇 준비 완료")
print("🤖 개선된 RAG 챗봇 (exit/quit/종료로 완전 종료)")
print("="*60)

try:
    _running = True
    while _running:
        try:
            q = input("\n❓ 질문: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\n👋 종료합니다.")
            _running = False
            break
        
        if q.lower() in ["exit","quit","종료"]:
            print("👋 종료합니다.")
            _running = False
            break
        
        if not q:
            continue
        
        try:
            print("\n처리 중...⏳")
            out = chat_once_improved(
                q, 
                vec_w=DEFAULT_VECTOR_WEIGHT, 
                bm25_w=DEFAULT_BM25_WEIGHT, 
                top_k=DEFAULT_TOP_K, 
                ctx_n=DEFAULT_CONTEXT_TOP_N, 
                max_ctx_chars=DEFAULT_CONTEXT_CHARS,
                debug=True
            )
            print("\n💬 답변:")
            print("-"*60)
            print(out)
            print("-"*60)
        except Exception as e:
            print(f"❌ 오류: {e}")
            # 오류가 나도 루프는 계속
finally:
    # PID 파일 삭제
    if os.path.exists(_CHATBOT_PID_FILE):
        os.remove(_CHATBOT_PID_FILE)
    print("\n✨ 챗봇이 완전히 종료되었습니다.")

✅ 개선된 챗봇 준비 완료
🤖 개선된 RAG 챗봇 (exit/quit/종료로 완전 종료)

처리 중...⏳
[질문] z

처리 중...⏳
[질문] z
[벡터 후보 수] 50
------------------------------------------------------------------------------------------
[Vector Top 10]
   1. 0.8182 | 홍삼액골드 | hongsamaeggoldeu-chunk-4
   2. 0.8149 | CAP 프로젝트 상세 (1기~5기 성과) | 3_3_1_cap_project_detail_phases1to5-chunk-3
   3. 0.8008 | 윤리경영 및 UNGC 참여 | 5_3_2_ethical_management_and_ungc-chunk-3
   4. 0.8005 | 알로에바이오틱스 | alroebaiotigseu-chunk-12
   5. 0.7980 | 이커머스 확장 스토리 상세 | 2_7_ecommerce_expansion_story_detail-chunk-2
   6. 0.7964 | 알로엔 더골드 플루이드 | alroen_deogoldeu_peulruideu-chunk-3
   7. 0.7911 | 독자 개발 핵심 성분 | 3_3_4_proprietary_ingredients_univestin_uniwhite-chunk-2
   8. 0.7882 | 알로엔 리바이탈 모델링 팩 | alroen_ribaital_modelring_paeg-chunk-2
   9. 0.7869 | W389 더마 브라이트닝 선크림 | w389_deoma_beuraiteuning_seonkeurim-chunk-4
  10. 0.7867 | 주요 브랜드 아이덴티티 | 4_3_brand_identity-chunk-1
------------------------------------------------------------------------------------------
[선택된 컨텍스트 수]

: 